# 浪子之心科技
# 此代码仅为演示wav2lip推理结束后用CodeFormer进行高清化的过程。
# 效果比Wav2Lip-GFPGAN好。


In [4]:
#@title 一、下载wav2lip源码和权重并安装wav2lip环境
#@title
!proxychains git clone https://github.com/zabique/Wav2Lip

#download the pretrained model
!proxychains wget 'https://iiitaphyd-my.sharepoint.com/personal/radrabha_m_research_iiit_ac_in/_layouts/15/download.aspx?share=EdjI7bZlgApMqsVoEUUXpLsBxqXbn5z8VTmoxp55YNDcIA' -O '/Wav2Lip/checkpoints/wav2lip_gan.pth'
a = !proxychains pip install https://raw.githubusercontent.com/AwaleSajil/ghc/master/ghc-1.0-py3-none-any.whl

# !pip uninstall tensorflow tensorflow-gpu
!cd Wav2Lip && proxychains pip install -r requirements.txt

#download pretrained model for face detection
!proxychains wget "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" -O "/content/Wav2Lip/face_detection/detection/sfd/s3fd.pth"

!proxychains pip install -q youtube-dl
!proxychains pip install ffmpeg-python
!proxychains pip install librosa==0.9.1

/bin/bash: line 1: cd: /content: No such file or directory
ProxyChains-3.1 (http://proxychains.sf.net)
Cloning into 'Wav2Lip'...
remote: Enumerating objects: 378, done.
remote: Total 378 (delta 0), reused 0 (delta 0), pack-reused 378
Receiving objects: 100% (378/378), 526.98 KiB | 1.88 MiB/s, done.
Resolving deltas: 100% (205/205), done.
ProxyChains-3.1 (http://proxychains.sf.net)
/content/Wav2Lip/checkpoints/wav2lip_gan.pth: No such file or directory
ProxyChains-3.1 (http://proxychains.sf.net)
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
^C
ERROR: Operation cancelled by user
ProxyChains-3.1 (http://proxychains.sf.net)
/content/Wav2Lip/face_detection/detection/sfd/s3fd.pth: No such file or directory
ProxyChains-3.1 (http://proxychains.sf.net)
ProxyChains-3.1 (http://proxychains.sf.net)
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
ProxyChains-3.1 (http://proxychains.sf.net)
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [2]:
#@title 二、下载CodeFormer的源码权重并安装依赖

# 下载CodeFormer的源码
%cd /content
!rm -rf CodeFormer
!proxychains git clone https://github.com/sczhou/CodeFormer.git
%cd /content/CodeFormer

[Errno 2] No such file or directory: '/content'
/data/disk1/wangshuo


ProxyChains-3.1 (http://proxychains.sf.net)
Cloning into 'CodeFormer'...
remote: Enumerating objects: 594, done.
remote: Counting objects: 100% (283/283), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 594 (delta 201), reused 182 (delta 182), pack-reused 311
Receiving objects: 100% (594/594), 17.31 MiB | 7.25 MiB/s, done.
Resolving deltas: 100% (288/288), done.
[Errno 2] No such file or directory: '/content/CodeFormer'
/data/disk1/wangshuo


In [ ]:
# 下载下载CodeFormer的权重
%cd /content/CodeFormer
!python scripts/download_pretrained_models.py facelib
!python scripts/download_pretrained_models.py CodeFormer

/content/CodeFormer
Downloading: "https://github.com/sczhou/CodeFormer/releases/download/v0.1.0/detection_Resnet50_Final.pth" to /content/CodeFormer/weights/facelib/detection_Resnet50_Final.pth

100% 104M/104M [00:00<00:00, 298MB/s] 
Downloading: "https://github.com/sczhou/CodeFormer/releases/download/v0.1.0/parsing_parsenet.pth" to /content/CodeFormer/weights/facelib/parsing_parsenet.pth

100% 81.4M/81.4M [00:00<00:00, 278MB/s]
Downloading: "https://github.com/sczhou/CodeFormer/releases/download/v0.1.0/codeformer.pth" to /content/CodeFormer/weights/CodeFormer/codeformer.pth

100% 359M/359M [00:02<00:00, 167MB/s]


In [ ]:
# 安装CodeFormer的依赖
%cd /content/CodeFormer
!python basicsr/setup.py develop

/usr/local/lib/python3.10/dist-packages/setuptools/__init__.py:84: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
!!

        ********************************************************************************
        Requirements should be satisfied by a PEP 517 installer.
        If you are using pip, you can try `pip install --use-pep517`.
        ********************************************************************************

!!
  dist.fetch_build_eggs(dist.setup_requires)
running develop
/usr/local/lib/python3.10/dist-packages/setuptools/command/develop.py:40: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://github.com/pypa/setuptools/issues/917 for details.
 

In [4]:
# 用wav2lip进行推理
# checkpoints 可以用wav2lip.pth 或者wav2lip_gan.pt
!python   --checkpoint_path checkpoints/wav2lip_gan.pth --face /content/mz_ready2_20s.mp4 --audio /content/mz_gen.wav

Using cuda for inference.
Traceback (most recent call last):
  File "/data/disk1/wangshuo/Wav2Lip/inference.py", line 280, in <module>
    main()
  File "/data/disk1/wangshuo/Wav2Lip/inference.py", line 183, in main
    raise ValueError('--face argument must be a valid path to video/image file')
ValueError: --face argument must be a valid path to video/image file


In [12]:
# 将Wav2Lip生成的视频转化成图片保存到images
import cv2
from tqdm import tqdm
from os import path
import os

vidcap = cv2.VideoCapture("../content/mz_ready2_gen.mp4")
number_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = vidcap.get(cv2.CAP_PROP_FPS)
print("FPS: ", fps, "Frames: ", number_frames)

for frameNumber in tqdm(range(number_frames)):
    _,image = vidcap.read()
    cv2.imwrite(path.join('../content/mid_images', str(frameNumber).zfill(4)+'.jpg'), image)

FPS:  30.0 Frames:  2551


  0%|          | 8/2551 [00:00<01:46, 23.91it/s]

100%|██████████| 2551/2551 [01:17<00:00, 32.95it/s]


In [13]:
#@title 五、用CodeFormer进行人脸修复

import cv2
import matplotlib.pyplot as plt
def display(img1, img2):
  fig = plt.figure(figsize=(25, 10))
  ax1 = fig.add_subplot(1, 2, 1)
  plt.title('Input', fontsize=16)
  ax1.axis('off')
  ax2 = fig.add_subplot(1, 2, 2)
  plt.title('CodeFormer', fontsize=16)
  ax2.axis('off')
  ax1.imshow(img1)
  ax2.imshow(img2)
def imread(img_path):
  img = cv2.imread(img_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img

In [8]:
## 只修复人脸
# CODEFORMER_FIDELITY = 0.5
# !python inference_codeformer.py -w $CODEFORMER_FIDELITY --has_aligned --input_path /content/images

# whole images ##整体图片修复
CODEFORMER_FIDELITY = 0.7
!proxychains python CodeFormer/inference_codeformer.py -w $CODEFORMER_FIDELITY --input_path /content/mz_ready2_gen.mp4 --output_path /content --bg_upsampler realesrgan

ProxyChains-3.1 (http://proxychains.sf.net)
/content/mz_ready2_gen.mp4: No such file or directory
Traceback (most recent call last):
  File "/data/disk1/wangshuo/CodeFormer/inference_codeformer.py", line 97, in <module>
    vidreader = VideoReader(args.input_path)
  File "/data/disk1/wangshuo/CodeFormer/basicsr/utils/video_util.py", line 43, in __init__
    meta = get_video_meta_info(video_path)
  File "/data/disk1/wangshuo/CodeFormer/basicsr/utils/video_util.py", line 19, in get_video_meta_info
    probe = ffmpeg.probe(video_path)
  File "/data/disk1/wangshuo/miniconda3/envs/GPTSoVits/lib/python3.9/site-packages/ffmpeg/_probe.py", line 23, in probe
    raise Error('ffprobe', out, err)
ffmpeg._run.Error: ffprobe error (see stderr output for detail)


In [22]:
# 生成修复后的新视频
import os
restoredFramesPath = '../content/enhanced_images/final_results/'
processedVideoOutputPath = '../content/final'

dir_list = os.listdir(restoredFramesPath)
dir_list.sort()

import cv2
import numpy as np
from tqdm import tqdm

img_array = []
print("processing ")
for filename in tqdm(dir_list):
    filename = restoredFramesPath+filename;
    img = cv2.imread(filename)
    if img is None:
      continue
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)
out = cv2.VideoWriter(processedVideoOutputPath+'/final.avi',cv2.VideoWriter_fourcc(*'DIVX'), 30, size)
for i in tqdm(range(len(img_array))):
  out.write(img_array[i])@@
out.release()

processing 


100%|██████████| 2551/2551 [09:13<00:00,  4.60it/s]


In [25]:
import ffmpeg
# 音视频合并 Generate audio video
audio = ffmpeg.input(f'../content/mz_gen.wav')
video = ffmpeg.input(f'../content/final/final.avi')
print("合并视音频")
out = ffmpeg.output(video, audio, f'../content/final/final_result.mp4')
out.run()
print("恭喜您，音视频合并完成，存放在output/final.mp4")

合并视音频


ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/tmp/build/80754af9/ffmpeg_1587154242452/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placeho --cc=/tmp/build/80754af9/ffmpeg_1587154242452/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100


恭喜您，音视频合并完成，存放在output/final.mp4


[aac @ 0x5555b1fc6940] Qavg: 3351.665
